# Test Different MLP
Here we will test differet versions. Maybe running the model multiple times and averaging the results. 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
import seaborn as sns
sns.set()

seed_val = 97

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [ ]:
def split_data(dataset, seed, train_ratio=0.6, shuffle=True):
    if shuffle:
        dataset = dataset.sample(frac=1, random_state=seed)#.reset_index(drop=True)

    train_dataset = dataset.sample(frac=train_ratio, random_state=0)
    test_dataset = dataset.drop(train_dataset.index)
    
    return train_dataset, test_dataset

def add_bias(data):
    N1 = np.shape(data)[0]
    N2 = np.shape(data)[1]
    a = -1*np.ones((N1,N2+1))
    a[:,:-1] = data
    return a

def add_noise(dataset, target_column=4, noise_var=0.01, input_n=False, output_n=False):
    """ Called on DATAFRAME training data. """
    features = dataset.to_numpy()[:,0:target_column]
    labels = np.reshape(dataset.to_numpy()[:,target_column], (-1,1))
    
    if input_n:
        noise = np.reshape(np.random.normal(0,0.01,np.shape(features)[0]*np.shape(features)[1]),(np.shape(features)[0],np.shape(features)[1]))
        features = features + noise 

    if output_n:
        noise = np.reshape(np.random.normal(0,0.01,np.shape(labels)[0]*np.shape(labels)[1]),(np.shape(labels)[0],np.shape(labels)[1]))
        labels = labels + noise
    return features, labels

In [ ]:
name = "LIM_scaled.csv"
name = "data.csv"

dataset = pd.read_csv(name)
# data2.describe().transpose()
dataset.pop("Unnamed: 0")

# scale the data
scaled_dataset = dataset.copy()
scaled_dataset['b1'] = MaxAbsScaler().fit_transform(dataset['b1'].values.reshape(-1,1))
scaled_dataset['a2'] = MaxAbsScaler().fit_transform(dataset['a2'].values.reshape(-1,1))
scaled_dataset['RatioTotalArea'] = MaxAbsScaler().fit_transform(dataset['RatioTotalArea'].values.reshape(-1,1))
scaled_dataset['frac'] = MaxAbsScaler().fit_transform(dataset['frac'].values.reshape(-1,1))

# split the data
train_dataset, test_dataset = split_data(scaled_dataset.copy(), seed_val, train_ratio=0.8)

# sort the data to check representation
sorted_train = train_dataset.sort_index()
sorted_test = test_dataset.sort_index()

# check representation
fig = plt.figure()
fig, ax = plt.subplots(ncols=2, figsize=(15,7))

ax[0].scatter(x=np.arange(len(sorted_train)), y=sorted_train['eta c'], marker='.', alpha=0.4)
ax[1].scatter(x=np.arange(len(sorted_test)), y=sorted_test['eta c'], marker='.', alpha=0.4)
ax[0].set_ylabel("eta c")
ax[0].set_title("Training Data")
ax[1].set_title("Testing Data")


ML Algo class.

In [ ]:
class MLP_Regression():
    def __init__(self, input_shape, nodes=30, dropout=False, rate=0.2, seed=97):
        self.model = Sequential()
        self.model.add(Dense(nodes, input_shape=(input_shape,), activation="relu", use_bias=True))
        if dropout:
            self.add_dropout(rate, seed)

    def learning_rate(self, initial_lr=1e-2, decay_steps=1e5, decay_rate=0.9):
        self.lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=initial_lr, decay_steps=decay_steps, decay_rate=decay_rate)

    def optimizer(self, momentum=0.1):
        self.optimizer = keras.optimizers.SGD(learning_rate=self.lr_schedule, momentum=momentum, nesterov=False)

    def add_layer(self, num_nodes):
        self.model.add(Dense(num_nodes, activation="relu", use_bias=True))
    
    def add_output_layer(self, out_nodes):
        self.model.add(Dense(out_nodes, activation="relu", use_bias=True))

    def add_dropout(self, rate, seed):
        self.model.add(keras.layers.Dropout(rate=rate, seed=seed))

    def compileModel(self, optimizer, loss="mean_squared_error", metrics=["MSE","MAE", "MAPE"]):
        self.model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    def train_model(self, X, Y, early_stopping=True, patience=3, epochs=10, batch_size=500, vali_split=0.2):
        if early_stopping:
            earlystop_callback = EarlyStopping(monitor="loss", min_delta=0, patience=patience, mode="min", restore_best_weights=True)
            hist = self.model.fit(X, Y, epochs=epochs, batch_size=batch_size, validation_split=vali_split, callbacks=[earlystop_callback])
            self.history = pd.DataFrame(hist.history)
            self.history['epoch'] = hist.epoch
        else:
            hist = self.model.fit(X, Y, epochs=epochs, batch_size=batch_size, validation_split=vali_split)
            self.history = pd.DataFrame(hist.history)
            self.history['epoch'] = hist.epoch

    def show_training_errors(self):
        hist = self.history

        fig1 = plt.figure()
        fig1, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,12))

        ax[0][0].set_xlabel('Epoch')
        ax[0][0].set_ylabel('MSE')
        ax[0][0].plot(hist['epoch'], hist['MSE'], label='Train Error')
        ax[0][0].plot(hist['epoch'], hist['val_MSE'], label='Val Error')
        ax[0][0].legend()
        ax[0][0].set_title("MSE Error")

        ax[0][1].set_xlabel('Epoch')
        ax[0][1].set_ylabel('MAE')
        ax[0][1].plot(hist['epoch'], hist['MAE'], label='Train Error')
        ax[0][1].plot(hist['epoch'], hist['val_MAE'], label='Val Error')
        ax[0][1].legend()
        ax[0][1].set_title("MAE Error")

        ax[1][0].set_xlabel('Epoch')
        ax[1][0].set_ylabel('MAPE')
        ax[1][0].plot(hist['epoch'], hist['MAPE'], label='Mean Abs {} Error'.format("%"))
        ax[1][0].legend()
        ax[1][0].set_title("MAPE")

    def test(self, test_features, test_labels):
        test_input = test_features
        test_output = np.reshape(test_labels, (-1,1))
        prediction = self.model.predict(test_input)

        fig2 = plt.figure()
        plt.plot(np.arange(len(test_output)), test_output, label="TestData")
        plt.plot(np.arange(len(prediction)), prediction, label="Prediction", alpha=0.4)
        plt.legend()
        plt.title("Model Prediction")

Organise the data for training. Notably, the eta c values > 1.2 correspond to frac values => 0.8 with the largest (3) b1 values and the 0.025 to 0.15 a2 values. 

In [ ]:
train_features = train_dataset.to_numpy()[:,0:4]
train_labels = train_dataset.to_numpy()[:,4]

# for noisey TRAINING data use this instead 
# train_features, train_labels = add_noise(train_dataset, noise_var=0.001, output_n=True)

sorted_test = test_dataset.sort_index()
test_features = sorted_test.to_numpy()[:,0:4]
test_labels = sorted_test.to_numpy()[:,4]

X = train_features.copy()
Y = train_labels.copy()

Start tests here.

In [ ]:
# define input and output dimensions
out_nodes = 1
in_features = 4 

summary = pd.DataFrame()
final_metrics = []

for i in range(0,10):
    # setup algo
    my_model = MLP_Regression(in_features, nodes=30, dropout=False, rate=0.1)
    my_model.learning_rate(initial_lr=1e-2, decay_steps=1e5, decay_rate=0.9)
    my_model.optimizer(momentum=0.1)

    # add layers
    my_model.add_layer(23)
    my_model.add_output_layer(out_nodes)
    # my_model.add_dropout(0.1, seed=seed_val)

    # run algo
    my_model.compileModel(my_model.optimizer)
    my_model.train_model(X,Y, epochs=200)

    final_metrics.append(my_model.history.iloc[[-1]])

summary = pd.concat(final_metrics)

In [ ]:
summary

In [ ]:
summary.to_csv("/media/nirav/34E0-F309/KTH/Thesis/ModelSummaries/DropOut_01.csv")

4-2-1 was bad. 
Dropout=0.1, no epochs reached 50.

In [ ]:
fig =  plt.figure()

# original
load_data = pd.read_csv("/media/nirav/34E0-F309/KTH/Thesis/ModelSummaries/Reg5.csv")
x = load_data[load_data["epoch"] > 50]["val_MAPE"].to_numpy()
plt.plot(np.arange(0,len(x)), x, label='Original', marker='o')

# Test 1, Changed 1h nodes
load_data = pd.read_csv("/media/nirav/34E0-F309/KTH/Thesis/ModelSummaries/Reg5_33.csv")
x = load_data[load_data["epoch"] > 50]["val_MAPE"].to_numpy()
plt.plot(np.arange(0,len(x)), x, label='33-23-1', marker='o')

# Test 2, Changed 1h nodes
load_data = pd.read_csv("/media/nirav/34E0-F309/KTH/Thesis/ModelSummaries/Reg5_27.csv")
x = load_data[load_data["epoch"] > 50]["val_MAPE"].to_numpy()
plt.plot(np.arange(0,len(x)), x, label='27-23-1', marker='o')

# Test 3, Changed momentum 0.1=>0.2
load_data = pd.read_csv("/media/nirav/34E0-F309/KTH/Thesis/ModelSummaries/Mom_02.csv")
x = load_data[load_data["epoch"] > 50]["val_MAPE"].to_numpy()
plt.plot(np.arange(0,len(x)), x, label='Original, momentum=0.2', marker='o')

# Test 4, Dropout on both, rate=0.1
# load_data = pd.read_csv("/media/nirav/34E0-F309/KTH/Thesis/ModelSummaries/DropOut_01.csv")
# x = load_data[load_data["epoch"] > 50]["val_MAPE"].to_numpy()
# plt.plot(np.arange(0,len(x)), x, label='Original, Dropout=0.1', marker='o')

# plt
plt.legend()
plt.ylabel("MAPE")
plt.xlabel("Iteration")
plt.title("MAPE over 10 sessions where # of Epochs was > 50")

In [ ]:
y = pd.concat(x)

In [ ]:
y